In [1]:
import torch
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorboard
import random
from tqdm import tqdm
import torchvision
import torchvision.datasets as datasets
from PIL import Image 
import torchvision.transforms as transforms

In [2]:
mnist_trainset = datasets.MNIST(root='./data', train=True, download=True, transform=None)
mnist_testset = datasets.MNIST(root='./data', train=False, download=True, transform=None)
print(mnist_trainset)
print(mnist_testset)
transform = transforms.Compose([ 
    transforms.PILToTensor() 
]) 
new_dataset=[]
test_data=[]
# transform = transforms.PILToTensor() 
# Convert the PIL image to Torch tensor 

for x in mnist_trainset:
    y=torch.zeros((1,10))
    y[0][x[1]]=1
    new_dataset.append((torch.reshape(transform(x[0])[0],(1,784))/255,y))

for x in mnist_testset:
    y=torch.zeros((1,10))
    y[0][x[1]]=1
    test_data.append((torch.reshape(transform(x[0])[0],(1,784))/255,y))

Dataset MNIST
    Number of datapoints: 60000
    Root location: ./data
    Split: Train
Dataset MNIST
    Number of datapoints: 10000
    Root location: ./data
    Split: Test


In [3]:
print(len(new_dataset))
val_size=(len(new_dataset))//10
val_set=[]
while len(val_set)<val_size:
    inst=random.randint(0,len(new_dataset)-1)
    if inst not in val_set:
        val_set.append(inst)

train_data=[]
val_data=[]

for i in range(0,len(new_dataset)):
    if i in val_set:
        val_data.append(new_dataset[i])
    else:
        train_data.append(new_dataset[i])

60000


In [4]:
epochs=1000
lr=0.1
batch_size=4
w=torch.rand((784,10))
b=torch.rand((10,))
best_w=w
best_b=b

In [5]:
def sigmoid(x):
    # print(x)
    # print(torch.exp(-x))
    # print(1+torch.exp(-x))
    return 1/(1+torch.exp(-x))

In [6]:
def softmax(x):
    e_x = torch.exp(x - torch.max(x))
    return e_x / e_x.sum()

In [7]:
def train_perceptron(X:torch.Tensor,W:torch.Tensor,b:torch.Tensor,y_true:torch.Tensor,lr:float):
    z=X@W+b
    y_pred=softmax(z)
    err=y_true-y_pred
    W_new=W+lr*x.T@err
    b_new=b+lr*err.mean(axis=0)
    return W_new,b_new

In [8]:
corr=0
for i in range(0,len(test_data)):
    x=test_data[i][0]
    y=test_data[i][1]
    y_pred=torch.argmax(sigmoid(x@best_w+best_b))
    if y_pred==torch.argmax(y):
        corr+=1
acc=corr/len(test_data)
print(acc)

0.0983


In [8]:
val_res=[]
max_val=0
scheduler=0
varia=tqdm(range(epochs))
for epoch in varia:
    i=0
    while i+batch_size-1<len(train_data):
        x=train_data[i][0]
        y=train_data[i][1]
        for j in range(1,batch_size):
            x=torch.cat((x,train_data[i+j][0]))
            y=torch.cat((y,train_data[i+j][1]))
        w,b=train_perceptron(x,w,b,y,lr)
        i+=batch_size
    corr=0
    for i in range(0,len(val_data)):
        x=val_data[i][0]
        y=val_data[i][1]
        y_pred=torch.argmax(sigmoid(x@w+b))
        if y_pred==torch.argmax(y):
            corr+=1
    acc=corr/len(val_data)
    scheduler+=1
    if acc>max_val:
        scheduler=0
        max_val=acc
        best_w=w
        best_b=b
    if scheduler==100:
        scheduler=0
        lr=lr/2
        
    
    val_res.append(acc)

100%|███████████████████████████████████████████████████████████████████████████████████████████████████| 1000/1000 [20:30<00:00,  1.23s/it]


In [9]:
print(max_val)

0.7971666666666667


In [15]:
corr=0
for i in range(0,len(test_data)):
    x=test_data[i][0]
    y=test_data[i][1]
    y_pred=torch.argmax(sigmoid(x@best_w+best_b))
    if y_pred==torch.argmax(y):
        corr+=1
acc=corr/len(test_data)

In [16]:
print(acc)

0.9173


In [11]:
torch.save(best_w,"w_best.pt")
torch.save(best_b,"b_best.pt")

In [14]:
best_w=torch.load("w_best.pt")
best_b=torch.load("b_best.pt")